### Kinetic Analysis of Alanine Dipeptide Simulation Datasets using VAMPnets and MSMs

In [4]:
# Import dependencies
import numpy as np
import matplotlib.pyplot as plt
import os

import mdtraj as md
from msmbuilder.featurizer import AtomPairsFeaturizer
from msmbuilder.cluster import KCenters
from msmbuilder.msm import MarkovStateModel

from sklearn.decomposition import PCA
from scipy.linalg import eigh

### Step 1 of MSM and VAMPnets: Generate Features

<img src="fig/alanine_dipeptide.png" width="500" align="center"/>

In [5]:
# Initialize the AtomPairsFeaturizer with atom pairs loaded from the 'ala2_atom_pairs' file
pairs_feat = AtomPairsFeaturizer(np.loadtxt("project2-md-dataset/ala2-xtc-1ps/ala2_atom_pairs"))

# Ensure the features directory exists
os.makedirs("project2-md-dataset/features", exist_ok=True)

# Process each trajectory file for indices 0 to 9
for i in range(10):
    # Load the trajectory with mdtraj
    traj = md.load(f"project2-md-dataset/ala2-xtc-1ps/ala2-1ps-0{i}.xtc", top="project2-md-dataset/ala2-xtc-1ps/ala2.pdb")
    # Transform the trajectory to features using the featurizer
    features = pairs_feat.transform([traj])
    # Save the features to a binary file in NumPy '.npy' format
    np.save(f"project2-md-dataset/features/ftraj_{i}", features[0])

# Repeat the process for indices 10 to 99
for i in range(10, 100):
    # Load the trajectory with mdtraj
    traj = md.load(f"project2-md-dataset/ala2-xtc-1ps/ala2-1ps-{i}.xtc", top="project2-md-dataset/ala2-xtc-1ps/ala2.pdb")
    # Transform the trajectory to features using the featurizer
    features = pairs_feat.transform([traj])
    # Save the features to a binary file in NumPy '.npy' format
    np.save(f"project2-md-dataset/features/ftraj_{i}", features[0])

# Assuming that the features directory contains all the feature files saved in the previous steps
feature_dir = "project2-md-dataset/features"
feature_files = [os.path.join(feature_dir, f) for f in os.listdir(feature_dir) if f.startswith('ftraj_') and f.endswith('.npy')]

# Load all the features into a list
features = [np.load(f) for f in feature_files]
print('Features:', len(features))

Features: 100


### Step 2 of MSM: Generate TICA

Autocorrelation Matrix $C_{00} = E_t [x_t x_t^T]$

Cross-Correlation Matrix $C_{01} = E_{t+\tau} [x_{t} x_{t+\tau}^T]$

Koopman Operator $K(\tau) = C_{00}^{-1}C_{01}$

**Objective of TICA**

$(\lambda, V) = \text{eig}[K(\tau)]$

$\chi_k(t) = \sum \Delta d_{(ij)}(t) \cdot V_{(ij),k}$


In [6]:

def compute_covariance_matrices(features, lag_time):
    """
    Compute the covariance and time-lagged covariance matrices.
    
    Parameters:
    - features: The input features, typically a list of 2D arrays.
    - lag_time: The lag time for the time-lagged covariance matrix.
    
    Returns:
    - C_0: Covariance matrix at time t.
    - C_tau: Time-lagged covariance matrix between t and t + tau.
    """
    # Concatenate all features to compute overall covariance
    concatenated_features = np.concatenate(features)
    # Center the features
    mean_features = np.mean(concatenated_features, axis=0)
    concatenated_features -= mean_features
    # Compute covariance matrix at time t
    C_0 = np.cov(concatenated_features.T)
    
    # Compute time-lagged covariance matrix
    concatenated_shifted_features = np.concatenate([f[lag_time:] for f in features])
    concatenated_shifted_features -= mean_features  # ensure the mean is removed
    C_tau = concatenated_features[:len(concatenated_shifted_features)].T @ concatenated_shifted_features / (len(concatenated_shifted_features) - 1)
    
    return C_0, C_tau

def perform_tica(C_0, C_tau, n_components):
    """
    Perform TICA given covariance matrices.
    
    Parameters:
    - C_0: Covariance matrix at time t.
    - C_tau: Time-lagged covariance matrix between t and t + tau.
    - n_components: Number of TICs to return.
    
    Returns:
    - eigenvalues: The top eigenvalues.
    - eigenvectors: The top eigenvectors corresponding to TICs.
    """
    # Solve the generalized eigenvalue problem
    eigenvalues, eigenvectors = eigh(C_tau, C_0)
    
    # Sort by largest eigenvalues
    idx = np.argsort(eigenvalues)[::-1]
    eigenvalues = eigenvalues[idx]
    eigenvectors = eigenvectors[:, idx]
    
    # Select the top components
    eigenvalues = eigenvalues[:n_components]
    eigenvectors = eigenvectors[:, :n_components]
    
    return eigenvalues, eigenvectors

# Define lag time and number of components you want to keep
lag_time = 1  # time steps 1ps
n_components = 3  # Number of TICs

# Assuming `features` is a list of arrays containing the pairwise distances of heavy atoms for each trajectory
C_0, C_tau = compute_covariance_matrices(features, lag_time)
eigenvalues, eigenvectors = perform_tica(C_0, C_tau, n_components)

# Print the top eigenvalues and eigenvectors
print('Eigenvalues:', eigenvalues)
print('Eigenvectors:', eigenvectors.shape)

# We now have the TICs, let's project the data onto the TIC space
# First, recalculate the mean features for the transformation step
mean_features = np.mean(np.concatenate(features), axis=0)

# Transform the original features into the TIC space for all trajectories
ttrajs = [(f - mean_features).dot(eigenvectors) for f in features]
print('ttrajs:', len(ttrajs))

Eigenvalues: [166.71749582  31.86559029  21.45451544]
Eigenvectors: (45, 3)
ttrajs: 100


### Step 3 of MSM: Generate a Micro State Model

In [7]:
# Initialize the K-Centers clustering algorithm with the desired number of clusters
kcenters = KCenters(n_clusters=400)

# Fit the K-Centers model to the TICA trajectories
kcenters.fit(ttrajs)

# Transform the trajectories into discrete states
ktrajs = kcenters.transform(ttrajs)

# Define a function to save the cluster assignments to files
def save_to_files(cluster_assignments, directory="project2-md-dataset/clusters"):
    # Ensure the directory exists
    os.makedirs(directory, exist_ok=True)
    
    # Save each trajectory's cluster assignments to a separate file
    for i, ktraj in enumerate(cluster_assignments):
        np.save(os.path.join(directory, f"ktraj_{i}"), ktraj)

# Call the function to save the cluster assignments to files
save_to_files(ktrajs)


### Step 4 of MSM: Build a Macro State Model


In [9]:
ktrajs_list = [np.load(f'project2-md-dataset/clusters/ktraj_{i}.npy') for i in range(100)]
ktrajs_array = np.array(ktrajs_list, dtype=int)

# Initialize the Markov State Model
msm = MarkovStateModel(lag_time=1)  # lag_time 1ps
msm.fit(ktrajs)

# Print out the timescales to check for convergence
print(msm.timescales_)


AttributeError: module 'numpy' has no attribute 'int'.
`np.int` was a deprecated alias for the builtin `int`. To avoid this error in existing code, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
The aliases was originally deprecated in NumPy 1.20; for more details and guidance see the original release note at:
    https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations

In [ ]:
# Plot the implied timescales
n_timescales = 5  # Or however many you wish to plot
timescales = msm.timescales_[:n_timescales]
plt.figure(figsize=(10, 4))
plt.plot(timescales, 'o-')
plt.title('Implied timescales')
plt.xlabel('Timescale index')
plt.ylabel('Timescale (lag units)')
plt.yscale('log')
plt.show()

# Plot the stationary distribution
plt.figure(figsize=(10, 4))
plt.bar(range(msm.n_states_), msm.populations_)
plt.title('Stationary Distribution')
plt.xlabel('State')
plt.ylabel('Population')
plt.show()